In [16]:
import os
import torch
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import default_loader
from PIL import Image
from scripts.resnet_model import get_resnet18

In [ ]:
img_size = 224
batch_size = 32
threshold = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "models/car_resnet18_model_best.pth"
unknown_dir = "data_split/test_unknown/unknown"
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])


In [22]:
class UnknownDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.image_paths = [
            os.path.join(folder, fname) for fname in os.listdir(folder)
            if fname.lower().endswith((".jpeg", ".jpg", ".png"))
        ]
        self.transform = transform
        self.loader = default_loader

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = self.loader(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(self.image_paths[idx])

In [23]:
unknown_dataset = UnknownDataset(unknown_dir, transform)
unknown_loader = DataLoader(unknown_dataset, batch_size=batch_size, shuffle=False)

class_names = sorted(os.listdir("data_split/train"))

def load_model(model_path, num_classes):
    model = get_resnet18(num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

model = load_model(model_path, len(class_names))

c:\Users\TUAN NGOC\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\TUAN NGOC\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
print(f"Predicting on {len(unknown_dataset)} unknown images\n")

Predicting on 549 unknown images



In [28]:
results = []

with torch.no_grad():
    for images, filenames in unknown_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)

        for i in range(images.size(0)):
            pred_class = class_names[preds[i].item()]
            confidence = probs[i][preds[i].item()]
            img_name = filenames[i]
            results.append((img_name, pred_class, confidence))

            if confidence < 0.5:
                print(f"{img_name}: Predicted: {pred_class}, Confidence: {confidence:.2f} -> Low Confidence")
            else:
                print(f"{img_name}: Predicted = {pred_class}, Confidence = {confidence:.2f}")


0.jpeg: Predicted: glass_shatter, Confidence: 0.34 -> Low Confidence
1003.jpeg: Predicted: head_lamp, Confidence: 0.28 -> Low Confidence
101.jpeg: Predicted: door_scratch, Confidence: 0.44 -> Low Confidence
1014.jpeg: Predicted = door_dent, Confidence = 0.66
1016.jpeg: Predicted: door_dent, Confidence: 0.46 -> Low Confidence
1027.jpeg: Predicted = glass_shatter, Confidence = 0.63
1029.jpeg: Predicted = tail_lamp, Confidence = 0.63
1030.jpeg: Predicted: door_dent, Confidence: 0.36 -> Low Confidence
1031.jpeg: Predicted: head_lamp, Confidence: 0.32 -> Low Confidence
1036.jpeg: Predicted = door_dent, Confidence = 0.87
1037.jpeg: Predicted: glass_shatter, Confidence: 0.43 -> Low Confidence
104.jpeg: Predicted: tail_lamp, Confidence: 0.20 -> Low Confidence
1040.jpeg: Predicted: tail_lamp, Confidence: 0.48 -> Low Confidence
1041.jpeg: Predicted = head_lamp, Confidence = 0.94
1046.jpeg: Predicted: door_dent, Confidence: 0.44 -> Low Confidence
1050.jpeg: Predicted = glass_shatter, Confidence =

Predicting on 549 unknown images

